In [56]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field

from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Literal

from dotenv import load_dotenv
import os

In [57]:
class SentimentSchema(BaseModel):
    sentiment: Literal["positive", "negative"]= Field(description="Provide the sentiment of the review")

In [58]:
class DiagnosisSchema(BaseModel):
    issue_type: Literal["UX", "Performance", "Bug", "Support", "Other"] = Field(description='The category of issue mentioned in the review')
    tone: Literal["angry", "frustrated", "disappointed", "calm"] = Field(description='The emotional tone expressed by the user')
    urgency: Literal["low", "medium", "high"] = Field(description='How urgent or critical the issue appears to be')


In [59]:
load_dotenv()

llm= HuggingFaceEndpoint(
    repo_id="meta-llama/Llama-3.2-3B-Instruct",
    task="text-generation",
    huggingfacehub_api_token= os.getenv("HUGGINGFACE_API_ENDPOINT"),
    max_new_tokens= 500,
    temperature= 0.3
)

model= ChatHuggingFace(llm= llm)

parser1= PydanticOutputParser(pydantic_object=SentimentSchema)
parser2= PydanticOutputParser(pydantic_object= DiagnosisSchema)

def getPrompt(review: str)-> str :
    prompt= PromptTemplate(
        # template= "Find the sentiment of the review {review} and the output should be strictly in the given format and it should not contain anything else except for the values present in the object:{format_instructions}",
        template= """
        You are a sentiment classifier.
        Analyze the following review and return the sentiment as JSON.

        Rules:
        - Sentiment must be exactly one of: "positive" or "negative".
        - Respond ONLY with valid JSON.
        - Do not include any text outside the JSON.

        Example output:
        {{"sentiment": "positive"}}

        Review: {review}
        """,
        input_variables=["review"],
        partial_variables={"format_instructions": parser1.get_format_instructions()}
    )
    
    return prompt.format(review= review)

def getDiagnosisPrompt(review: str) -> str:
    """
    Build a prompt for classifying a negative review using DiagnosisSchema.
    Uses parser2 to enforce JSON format.
    """
    prompt = PromptTemplate(
        template="""
        You are a customer support analyst AI.
        Analyze the following negative review and return a diagnosis as JSON.

        Rules:
        - issue_type must be exactly one of: "UX", "Performance", "Bug", "Support", "Other".
        - tone must be exactly one of: "angry", "frustrated", "disappointed", "calm".
        - urgency must be exactly one of: "low", "medium", "high".
        - Respond ONLY with valid JSON.
        - Do not include any text outside the JSON.

        Example output:
        {{"issue_type": "Performance", "tone": "frustrated", "urgency": "high"}}

        Review: {review}
        """,
        input_variables=["review"],
        partial_variables={"format_instructions": parser2.get_format_instructions()}
    )

    return prompt.format(review=review)
    

chain1= model | parser1
chain2= model | parser2

In [60]:
output= chain1.invoke(getPrompt("This is the best product I have seen in my entire life"))

print(output)

sentiment='positive'


In [61]:
class ReviewState(TypedDict):
    review: str
    sentiment: Literal["positive", "negative"]
    # diagnosis: {"issue_type", "tone", "urgency"}
    diagnosis: dict
    response: str

In [62]:
def find_sentiment(state:ReviewState)-> ReviewState:
    review= state['review']
    sentiment= chain1.invoke(getPrompt(review)).sentiment
    
    return {"sentiment":sentiment}

In [63]:
def checkSentiment(state:ReviewState)-> Literal["run_diagnosis", "positive_response"]:
    if state['sentiment']== 'positive':
        return 'positive_response'
    else:
        return 'run_diagnosis'

In [64]:
def positive_response(state:ReviewState)-> ReviewState:
    prompt= f"Write a warm response for the given positive response: {state['review']}. Also ask the user to leave the feedback on the website"
    
    response= model.invoke(prompt).content
    
    return {"response": response}

In [65]:
def run_diagnosis(state:ReviewState)-> ReviewState:
    response= chain2.invoke(getDiagnosisPrompt(state['review']))
    
    return {"diagnosis": response.model_dump()} # model dump is used to convert the json object into the dict

def negative_response(state: ReviewState)-> ReviewState:
    diagnosis= state['diagnosis']
    prompt = f"""You are a support assistant.
The user had a '{diagnosis['issue_type']}' issue, sounded '{diagnosis['tone']}', and marked urgency as '{diagnosis['urgency']}'.
Write an empathetic, helpful resolution message.
"""
    response= model.invoke(prompt).content
    
    return {"response": response}

In [66]:
graph= StateGraph(ReviewState)

graph.add_node('find_sentiment', find_sentiment)
graph.add_node('positive_response', positive_response)
graph.add_node('run_diagnosis', run_diagnosis)
graph.add_node('negative_response', negative_response)

graph.add_edge(START, 'find_sentiment')

graph.add_conditional_edges('find_sentiment', checkSentiment)

graph.add_edge('run_diagnosis', 'negative_response')

graph.add_edge('positive_response', END)
graph.add_edge('negative_response', END)

workflow= graph.compile()

In [67]:
initial_state={"review":"I LG Ultragear 32 inch monitor but it is not working properly. I am very much disappointed and not having any trust on this company. I will not suggest anyone to purchase from LG"}
final_state= workflow.invoke(initial_state)

print(final_state)

{'review': 'I LG Ultragear 32 inch monitor but it is not working properly. I am very much disappointed and not having any trust on this company. I will not suggest anyone to purchase from LG', 'sentiment': 'negative', 'diagnosis': {'issue_type': 'Performance', 'tone': 'disappointed', 'urgency': 'high'}, 'response': "Dear [User's Name],\n\nI want to start by apologizing for the 'Performance' issue that you encountered, and I'm truly sorry that it caused you disappointment. I can imagine how frustrating it must be when something doesn't work as expected, especially when you mark it as 'high' urgency.\n\nI've investigated the matter, and I'm happy to inform you that we've taken immediate action to resolve the issue. Our team has been working diligently to address the problem, and we've implemented a solution that should significantly improve the performance of the system.\n\nTo ensure a smooth experience for you, I'd like to offer the following:\n\n1. A temporary workaround has been imple